I'm going to share YET ANOTHER simple but working tsp solutions optimization technique.

The idea is straightforward: if you have THREE cities in the path that are 'close' (in distance terms) to each other you can try to split the path by them and try different permutations of resulting chunks hoping that the decrease of the total penalty in this part of the path will be greater than the increase of the length caused by the permutation itself. 

Also instead of just permutating the chunks of the path you can  try to revert the order of some chunks but I won't do it in this kernel.

1. Import all that we need.

In [5]:
import numpy as np
import pandas as pd
import numba
from sympy import isprime, primerange
from math import sqrt
from sklearn.neighbors import KDTree
from tqdm import tqdm
from itertools import combinations, permutations
from functools import lru_cache

2. Read input data and define some arrays that we'll need later.

In [6]:
cities = pd.read_csv('cities.csv', index_col=['CityId'])
XY = np.stack((cities.X.astype(np.float32), cities.Y.astype(np.float32)), axis=1)
is_not_prime = np.array([0 if isprime(i) else 1 for i in cities.index], dtype=np.int32)

3. Define fast scoring functions using numba.

In [7]:
@numba.jit('f8(i8, i8, i8)', nopython=True, parallel=False)
def cities_distance(offset, id_from, id_to):
    xy_from, xy_to = XY[id_from], XY[id_to]
    dx, dy = xy_from[0] - xy_to[0], xy_from[1] - xy_to[1]
    distance = sqrt(dx * dx + dy * dy)
    if offset % 10 == 9 and is_not_prime[id_from]:
        return 1.1 * distance
    return distance


@numba.jit('f8(i4, i8[:])', nopython=True, parallel=False)
def score_chunk(offset, chunk):
    pure_distance, penalty = 0.0, 0.0
    penalty_modulo = 9 - offset % 10
    for path_index in numba.prange(chunk.shape[0] - 1):
        id_from, id_to = chunk[path_index], chunk[path_index+1]
        xy_from, xy_to = XY[id_from], XY[id_to]
        dx, dy = xy_from[0] - xy_to[0], xy_from[1] - xy_to[1]
        distance = sqrt(dx * dx + dy * dy)
        pure_distance += distance
        if path_index % 10 == penalty_modulo and is_not_prime[id_from]:
            penalty += distance
    return pure_distance + 0.1 * penalty


@numba.jit('f8(i8[:])', nopython=True, parallel=False)
def score_path(path):
    return score_chunk(0, path)


@numba.jit
def chunk_scores(chunk):
    scores = np.zeros(10)
    pure_distance = 0
    for i in numba.prange(chunk.shape[0] - 1):
        id_from, id_to = chunk[i], chunk[i+1]
        xy_from, xy_to = XY[id_from], XY[id_to]
        dx, dy = xy_from[0] - xy_to[0], xy_from[1] - xy_to[1]
        distance = sqrt(dx * dx + dy * dy)
        pure_distance += distance
        if is_not_prime[id_from]:
            scores[9-i%10] += distance
    scores *= 0.1
    scores += pure_distance
    return scores

def score_compound_chunk(offset, head, chunks, tail, scores, indexes_permutation=None):
    if indexes_permutation is None:
        indexes_permutation = range(len(chunks))
    score = 0.0
    last_city_id = head
    for index in indexes_permutation:
        chunk, chunk_scores = chunks[index], scores[index]
        score += cities_distance(offset % 10, last_city_id, chunk[0])
        score += chunk_scores[(offset + 1) % 10]
        last_city_id = chunk[-1]
        offset += len(chunk)
    return score + cities_distance(offset % 10, last_city_id, tail)


4. Precompute close cities triplets using KDTree.

In [8]:
kdt = KDTree(XY)

In [9]:
triplets = set()
for city_id in tqdm(cities.index):
    dists, neibs = kdt.query([XY[city_id]], 7)
    for triplet in combinations(neibs[0], 3):
        if all(triplet):
            triplets.add(tuple(sorted(triplet)))
    neibs = kdt.query_radius([XY[city_id]], 10, count_only=False, return_distance=False)
    for triplet in combinations(neibs[0], 3):
        if all(triplet):
            triplets.add(tuple(sorted(triplet)))

print(f'{len(triplets)} cities triplets are selected.')

# sort triplets by distance
@numba.jit('f8(i8[:])', nopython=True, parallel=False)
def sum_distance(ids):
    res = 0
    for i in numba.prange(len(ids)):
        for j in numba.prange(i + 1, len(ids)):
            res += cities_distance(0, ids[i], ids[j])
    return res

triplets = np.array(list(triplets))
distances = np.array(list(map(sum_distance, tqdm(triplets))))
order = distances.argsort()
triplets = triplets[order]

100%|██████████| 197769/197769 [01:42<00:00, 1930.59it/s]


9355215 cities triplets are selected.


100%|██████████| 9355215/9355215 [00:06<00:00, 1397094.44it/s]


[](http://)5. Load the initial path to start optimization from. I'll use the one generated by [my previous kernel](https://www.kaggle.com/kostyaatarik/close-ends-chunks-optimization-aka-2-opt).

In [10]:
path = np.array(pd.read_csv('submissions/mip_LKH_1_42_n250_t1000_s0.csv').Path)

6. Use optimization described above.

In [11]:
def not_trivial_permutations(iterable):
    perms = permutations(iterable)
    next(perms)
    yield from perms


@lru_cache(maxsize=None)
def not_trivial_indexes_permutations(length):
    return np.array([list(p) for p in not_trivial_permutations(range(length))])

path_index = np.argsort(path[:-1])
print(f'Total score is {score_path(path):.2f}.')
for _ in range(3):
    for ids in tqdm(triplets):
        i, j, k = sorted(path_index[ids])
        head, tail = path[i-1], path[k+1]
        chunks = [path[i:i+1], path[i+1:j], path[j:j+1], path[j+1:k], path[k:k+1]]
        chunks = [chunk for chunk in chunks if len(chunk)]
        scores = [chunk_scores(chunk) for chunk in chunks]
        default_score = score_compound_chunk(i-1, head, chunks, tail, scores)
        best_score = default_score
        for indexes_permutation in not_trivial_indexes_permutations(len(chunks)):
            score = score_compound_chunk(i-1, head, chunks, tail, scores, indexes_permutation)
            if score < best_score:
                permutation = [chunks[i] for i in indexes_permutation]
                best_chunk = np.concatenate([[head], np.concatenate(permutation), [tail]])
                best_score = score
        if best_score < default_score:
            path[i-1:k+2] = best_chunk
            path_index = np.argsort(path[:-1])
            print(f'New total score is {score_path(path):.2f}. Permutating path at indexes {i}, {j}, {k}.')
    triplets = triplets[:10**6]

  0%|          | 0/9355215 [00:00<?, ?it/s]

Total score is 1515471.13.


  0%|          | 506/9355215 [00:00<396:19:39,  6.56it/s]

New total score is 1515470.99. Permutating path at indexes 29716, 29717, 29727.


  0%|          | 2299/9355215 [00:01<33:42:32, 77.07it/s]

...

New total score is 1515411.40. Permutating path at indexes 163079, 163080, 163086.


 81%|████████▏ | 7605405/9355215 [2:26:16<57:41, 505.49it/s]

New total score is 1515410.88. Permutating path at indexes 43639, 43646, 43651.


 83%|████████▎ | 7758733/9355215 [2:30:57<51:11, 519.74it/s]

KeyboardInterrupt: 

In [12]:
print(f'Final score is {score_path(path):.2f}.')

Final score is 1515410.88.


7. Save the result path.

In [13]:
def make_submission(name, path):
    pd.DataFrame({'Path': path}).to_csv(f'{name}.csv', index=False)

 83%|████████▎ | 7758733/9355215 [2:31:16<51:11, 519.74it/s]

In [14]:
make_submission("3.5_after_mip", path)